# Ensemble Models

-- It's better to use models that are essentially different, as the weakness in one balances the other one.

-- If all models can estimate class probability, then you can set *voting=soft*. This often performs better because it gives more weight to confident scores.

-- Another way to ensemble models is to train the same model with different samples from you training data. Here we have *bagging (bootstrap aggregating, with replacament)* and *pasting (without replacement)*

-- The aggregation function is typically the mode (most frequent value) or the average (for regression)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

logreg = LogisticRegression()
forest = RandomForestClassifier()
svm = SVC()

voting_classifier = VotingClassifier(
    estimators=[("linear", logreg), ("forest", forest), ("svm", svm)],
    voting="hard"
)

# fit the classifier
voting_classifier.fit(X_train,y_train)

# check each classifier accuracy on the test set
from sklearn.metrics import accuracy_score

for score in (logreg, forest, svm, voting_classifier):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(score.__class__.__name__, accuracy_score(y_pred, y_test))